In [28]:
import pandas as pd 
import hopsworks 
from datetime import datetime, timedelta

import numpy as np



#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()

True

In [29]:
api_key = os.environ.get('HOPSWORKS_API')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()
mr = project.get_model_registry() 

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [30]:
start_date = datetime.now() - timedelta(hours=48)
print(start_date.strftime("%Y-%m-%d"))

2024-10-14


In [31]:
end_date = datetime.now() - timedelta(hours=24)
print(end_date.strftime("%Y-%m-%d"))

2024-10-15


In [36]:
feature_view = fs.get_feature_view('amd_stock_fv', 12)
feature_view.init_batch_scoring(training_dataset_version=1)

In [37]:
df = feature_view.get_batch_data(read_options={"use_hive": True})

2024-10-16 18:51:12,199 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (2.68s) 


In [38]:
df.head()

,date,f_1__open
0,2001-02-27,22.82
1,2018-02-23,11.95
2,2005-01-21,15.64
3,2003-11-24,17.15
4,2002-12-13,7.38


In [39]:
import joblib

the_model = mr.get_model("ProphetModel", version=8)
model_dir = the_model.download()

model = joblib.load(model_dir + "/Prophet_model.pkl")

In [40]:
print("Columns in df:", df.columns.tolist())

Columns in df: ['date', 'f_1__open']


In [41]:
# Remove timezone information from 'ds' column
df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)


In [42]:
print(df['date'].dtype)


datetime64[ns]


In [43]:
df.head()

,date,f_1__open
0,2001-02-27,22.82
1,2018-02-23,11.95
2,2005-01-21,15.64
3,2003-11-24,17.15
4,2002-12-13,7.38


In [44]:
prophet_df = df[['date', 'f_1__open']].rename(columns={'date': 'ds', 'f_1__open': 'y'})

In [45]:
prophet_df.head()

,ds,y
0,2001-02-27,22.82
1,2018-02-23,11.95
2,2005-01-21,15.64
3,2003-11-24,17.15
4,2002-12-13,7.38


In [46]:
# Predict using the model
predictions = model.predict(prophet_df)

# Display predictions
print(predictions)


             ds       trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0    1999-11-01 -464.295697 -392.746707 -372.256450  -464.295697  -464.295697   
1    1999-11-02 -464.228626 -283.782939 -264.079089  -464.228626  -464.228626   
2    1999-11-03 -464.161554 -331.060875 -310.457362  -464.161554  -464.161554   
3    1999-11-04 -464.094483 -622.163310 -602.131879  -464.094483  -464.094483   
4    1999-11-05 -464.027412 -362.903345 -343.604928  -464.027412  -464.027412   
...         ...         ...         ...         ...          ...          ...   
6274 2024-10-09   -6.629561  131.250238  157.874328   -14.072656     1.676959   
6275 2024-10-10   -6.599396 -161.185784 -135.513520   -14.091255     1.776290   
6276 2024-10-11   -6.569231   96.887035  123.344031   -14.103685     1.843268   
6277 2024-10-14   -6.478736   69.918355   96.250887   -14.052151     1.961456   
6278 2024-10-15   -6.448571  176.819973  202.664118   -14.070504     2.034815   

      additive_terms  addit

In [47]:
predictions 

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,daily,...,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,1999-11-01,-464.295697,-392.746707,-372.256450,-464.295697,-464.295697,82.129560,82.129560,82.129560,40.46756,...,42.609737,42.609737,42.609737,-0.947737,-0.947737,-0.947737,0.0,0.0,0.0,-382.166138
1,1999-11-02,-464.228626,-283.782939,-264.079089,-464.228626,-464.228626,190.014100,190.014100,190.014100,40.46756,...,149.778556,149.778556,149.778556,-0.232017,-0.232017,-0.232017,0.0,0.0,0.0,-274.214526
2,1999-11-03,-464.161554,-331.060875,-310.457362,-464.161554,-464.161554,143.744250,143.744250,143.744250,40.46756,...,102.671902,102.671902,102.671902,0.604788,0.604788,0.604788,0.0,0.0,0.0,-320.417305
3,1999-11-04,-464.094483,-622.163310,-602.131879,-464.094483,-464.094483,-148.399492,-148.399492,-148.399492,40.46756,...,-190.405674,-190.405674,-190.405674,1.538622,1.538622,1.538622,0.0,0.0,0.0,-612.493975
4,1999-11-05,-464.027412,-362.903345,-343.604928,-464.027412,-464.027412,111.034621,111.034621,111.034621,40.46756,...,68.025288,68.025288,68.025288,2.541773,2.541773,2.541773,0.0,0.0,0.0,-352.992791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6274,2024-10-09,-6.629561,131.250238,157.874328,-14.072656,1.676959,150.483465,150.483465,150.483465,40.46756,...,102.671902,102.671902,102.671902,7.344003,7.344003,7.344003,0.0,0.0,0.0,143.853905
6275,2024-10-10,-6.599396,-161.185784,-135.513520,-14.091255,1.776290,-142.559879,-142.559879,-142.559879,40.46756,...,-190.405674,-190.405674,-190.405674,7.378234,7.378234,7.378234,0.0,0.0,0.0,-149.159275
6276,2024-10-11,-6.569231,96.887035,123.344031,-14.103685,1.843268,115.761383,115.761383,115.761383,40.46756,...,68.025288,68.025288,68.025288,7.268535,7.268535,7.268535,0.0,0.0,0.0,109.192152
6277,2024-10-14,-6.478736,69.918355,96.250887,-14.052151,1.961456,89.187866,89.187866,89.187866,40.46756,...,42.609737,42.609737,42.609737,6.110569,6.110569,6.110569,0.0,0.0,0.0,82.709130


In [48]:
api_key = os.environ.get('HOPSWORKS_API')
project = hopsworks.login(api_key_value=api_key)
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.


In [49]:
results_fg = fs.get_or_create_feature_group(
    name= 'stock_prediction_results',
    version = 3,
    description = 'Predction of AMD close stock price',
    primary_key = ['y'],
    event_time = ['ds'],
    online_enabled = False,
)

2024-10-16 18:52:39,141 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.



In [50]:
results_fg.insert(prophet_df, write_options={"wait_for_job" : False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/564374/fs/560197/fg/1285459


Uploading Dataframe: 0.00% |          | Rows 0/6279 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: stock_prediction_results_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/stock_prediction_results_3_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x16e8929de80>, None)